In [1]:
pwd

'/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/PARAMETRIC/notebook'

In [9]:
cd /mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/NASTRAN

/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/NASTRAN


/mnt/c/Users/nakagawa/Desktop/python_venv/feniax/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [15]:
import numpy as np

In [11]:
import os
from .PARAMETRIC.pch_parser import *

ModuleNotFoundError: No module named 'PARAMETRIC'

In [18]:
NASTRAN_DIR="cmd.exe /c C:/MSC.Software/MSC_Nastran/20182/bin/nast20182.exe"
fname='./BUG_103cap.bdf'
dir_output='./'
command=f'{NASTRAN_DIR} {fname} out={dir_output} old=no news=no'
os.system(command)

MSC Nastran V2018.2 (Intel Windows 10 Pro 6.2 9200) Wed Jul  3 22:47:09 2024
MSC Nastran beginning job bug_103cap.



*** SYSTEM INFORMATION MESSAGE (pgm: nastran, fn: estimate_job_requirements)
    Starting ESTIMATE, please wait...

*** USER INFORMATION MESSAGE (pgm: nastran, fn: estimate_job_requirements)
    Estimated memory=65370.0MB
    Estimated bpool=16342.5MB


MSC Nastran started  c:\msc.software\msc_nastran\20182\msc20182\win64i8\mscdate
MSC Nastran finished c:\msc.software\msc_nastran\20182\msc20182\win64i8\mscdate
MSC Nastran job bug_103cap completed.


0

In [19]:
_,Ka,Ma=read_pch('./BUG_103cap.pch')

In [31]:
np.where(np.abs(Ma).sum(axis=1)==0.0)[0]//6

array([46, 85])

In [24]:
from pyNastran.bdf.bdf import read_bdf

In [26]:
bdfmodel=read_bdf('./BUG_103cap.bdf',debug=None)

In [32]:
np.sort(bdfmodel.asets[0].node_ids)[np.where(np.abs(Ma).sum(axis=1)==0.0)[0]//6]

array([    4001, 10004001])

In [23]:
print(279//6,)

46